# Dependencies 

In [ ]:
# setup
!gdown 1oJh0U3g_bUx6UPX4xix2UHMVHeCE_H1y
!unzip -q hvb.zip
!mv content/data /content/
!rm -r /content/content

!gdown 1a0EGlsLbXnGn1xwZoSqT0tcdAQ1L2nfd # train.py
!gdown 1yCmjRbxXRxfEN5LXdnE1Zpl8ZOIzdrAO # train.yaml
!gdown 1KHmdcLVFI9ontvGmi5J6vfaropGYuKcr # inference.yaml

!pip install speechbrain -q

In [ ]:
import speechbrain as sb
from speechbrain.pretrained import EncoderDecoderASR
import json
import torchaudio
import torch
from torch import nn
from tqdm import tqdm
from collections import Counter
from IPython.display import Audio
from scipy.io import wavfile

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
crdnn = EncoderDecoderASR.from_hparams(
    source='speechbrain/asr-crdnn-rnnlm-librispeech',
    savedir='asr-crdnn-rnnlm-librispeech',
    run_opts={'device': 'cuda'}
)

The manifests we prepared to use with SpeechBrain are jsons with the structure
```
{
    "15748": {
        "wav": "/content/data/segments/15748.wav",
        "length": 1.86,
        "words": "WHAT DAY WOULD YOU LIKE FOR YOUR APPOINTMENT"
    },
    ...
}
```

We first load them and define a function to batch them into a format that our `EncoderDecoderASR` object can ingest:

In [ ]:
TEST_SIZE = 500 # for faster processing

with open('data/test_manifest.json', 'r') as f:
    test_manifest = json.load(f)
test_manifest = {
    k: v for k, v in list(test_manifest.items())[:TEST_SIZE]
}

def batchify(manifest, batch_size):
    keys = list(manifest.keys())
    wav_paths = list(map(lambda x: x['wav'], manifest.values()))
    iterable = zip(keys, wav_paths)
    num_examples = len(manifest)
    for i in range(0, num_examples, batch_size):
        batch_wavs = nn.utils.rnn.pad_sequence([
            torchaudio.load(path)[0].squeeze(0)
            for path in wav_paths[i:min(i + batch_size, num_examples)]
        ], batch_first=True)
        batch_keys = keys[i:min(i + batch_size, num_examples)]
        batch_wav_lens = torch.tensor([
            manifest[key]['length'] for key in batch_keys
        ])
        batch_wav_lens = batch_wav_lens / batch_wav_lens.max()
        yield batch_keys, batch_wavs, batch_wav_lens


In [ ]:
# test_manifest

Next, we feed our test examples through the pretrained ASR model:

In [ ]:
true_dict = {key: test_manifest[key]['words'] for key in test_manifest}

def inference(model, test_manifest, batch_size=8):
    torch.cuda.empty_cache()
    pred_dict = {}
    for keys, wavs, wav_lens in tqdm(batchify(test_manifest, batch_size), total=round(len(test_manifest) / batch_size + 0.5)):
        transcriptions, _ = model.transcribe_batch(wavs.to(device), wav_lens.to(device))
        for key, transcription in zip(keys, transcriptions):
            pred_dict[key] = transcription
    return pred_dict

pred_dict = inference(crdnn, test_manifest)

In [ ]:
# this data structure stores WER information we use later. 
details_by_utterance = sb.utils.edit_distance.wer_details_by_utterance(
    {k: v.split() for k, v in true_dict.items()},
    {k: v.split() for k, v in pred_dict.items()},
)

In [ ]:
# word error rate (WER) summary using data structure we just created
sb.utils.edit_distance.wer_summary(details_by_utterance)

In [ ]:
def summarize(detail_dict, true_dict, pred_dict):
    print(f"{detail_dict['key']}: {detail_dict['WER']}")
    print(f"\tTrue: {true_dict[detail_dict['key']]}")
    print(f"\tPred: {pred_dict[detail_dict['key']]}")

for wer_dict in sb.utils.edit_distance.top_wer_utts(details_by_utterance, 10)[0]:
    summarize(wer_dict, true_dict, pred_dict)

1130: 2400.0
	True: ALRIGHT
	Pred: RIPE RIGHT RIPE RIPE RIPE RIPE RIPE RIPE RIPE RIPE RIPE RIPE RIPE RIPE RIPE RIPE RIPE RIPE RIPE RIPE RIPE RIPE RIPE RIPE
3020: 737.5
	True: OKAY THANKS AND WHAT IS THE BILL AMOUNT
	Pred: OKAY THANKS AND WHAT IF YE DON'T AMOUNT THANKS AN WHAT IF YE DON'T AMOUNT THANKS AN WHAT IF YE BILL AMOUNT THINK AN WHAT IF YE DON'T AMOUNT THINK AN WHAT IF YE DON'T AMOUNT THINK AN WHAT IF YE DON'T AMOUNT THINK AN WHAT IF YE DON'T AMOUNT THINK AN WHAT IF YE DON'T AMOUNT THANKS AN WHAT IF YE BUILD A MOUNT
22147: 620.0
	True: WHAT IS THE TRANSFER AMOUNT
	Pred: OF THE TRANSFER AMOUNT AMOUNT AMOUNT AMOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT MOUNT
20613: 600.0
	True: BYE
	Pred: I DON'T EAT DRINK EAT DRINK
5390: 600.0
	True: NO
	Pred: NOW'S NOW NOW NOW NOW NOW
7385: 558.3333333333334
	True: NO I GOT ALL I NEED FOR TODAY THANK YOU SO MUCH
	Pred: NO I DON'T KNOW 

Seems that our predictions keep outputting the same word over and over. Let's see why.

Select some examples the model wrongly predicts, and try to build a hypothesis around what in the data is associated with the model making mistakes. Examples of mistake types include:
- Repeated wrong word
- A few correct words but clearly wrong transcript

Give examples of at least 3 audio files and different kinds of errors you have identified in these audio files. Add this to `README.txt`.


Here's a code snippet to help you get started. 

In [ ]:
example = details_by_utterance[14]
summarize(example, true_dict, pred_dict)
Audio(test_manifest[example['key']]['wav'])

5729: 150.0
	True: DESTINATION ACCOUNT
	Pred: DEFINITION A COW


(Hint: In our initial checks, poor audio quality seems to be associated with repeated word errors.)

In [ ]:
# this downloads the training and config files for our fine tuning setup
!gdown 1v_3Kl8OrUd6_1_D0ZGoYVFEuOKhZ7YMo # train.py
!gdown 17cQIpx5kLLMCD23EDaE0EYg2E9LPqMCF # train.yaml
!gdown 1CWYOD2PC97gXguW4krc9122HKAraHkYS # inference.yaml

Downloading...
From: https://drive.google.com/uc?id=1v_3Kl8OrUd6_1_D0ZGoYVFEuOKhZ7YMo
To: /content/train.py
100% 18.3k/18.3k [00:00<00:00, 24.1MB/s]
Downloading...
From: https://drive.google.com/uc?id=17cQIpx5kLLMCD23EDaE0EYg2E9LPqMCF
To: /content/train.yaml
100% 12.1k/12.1k [00:00<00:00, 16.0MB/s]
Downloading...
From: https://drive.google.com/uc?id=1CWYOD2PC97gXguW4krc9122HKAraHkYS
To: /content/inference.yaml
100% 4.96k/4.96k [00:00<00:00, 8.31MB/s]


In [ ]:
torch.cuda.empty_cache()

!python train.py train.yaml --batch_size=4
# OOM on batch_size=5

rirs_noises.zip: 1.31GB [00:49, 26.3MB/s]                
Extracting ./data/rirs_noises.zip to ./data
speechbrain.core - Beginning experiment!
speechbrain.core - Experiment folder: results/CRDNN_BPE_960h_LM/2602
speechbrain.pretrained.fetching - Fetch lm.ckpt: Delegating to Huggingface hub, source speechbrain/asr-crdnn-rnnlm-librispeech.
speechbrain.pretrained.fetching - Fetch tokenizer.ckpt: Delegating to Huggingface hub, source speechbrain/asr-crdnn-rnnlm-librispeech.
speechbrain.pretrained.fetching - Fetch asr.ckpt: Delegating to Huggingface hub, source speechbrain/asr-crdnn-rnnlm-librispeech.
speechbrain.utils.parameter_transfer - Loading pretrained files for: lm, tokenizer, model
speechbrain.core - Info: ckpt_interval_minutes arg from hparam file is used
speechbrain.core - 173.0M trainable parameters in ASR
speechbrain.utils.checkpoints - Would load a checkpoint here, but none found yet.
speechbrain.utils.epoch_loop - Going into epoch 1
  0% 0/3405 [00:00<?, ?it/s]/usr/local/lib/p

In [ ]:
ckpt_path = "/content/results/CRDNN_BPE_960h_LM/2602/save/CKPT+2023-04-04+14-24-27+00"
!cp inference.yaml {ckpt_path}

In [ ]:
device = 'cpu'
our_model = EncoderDecoderASR.from_hparams(
    source=ckpt_path, 
    hparams_file='inference.yaml', 
    savedir="our_ckpt",
    run_opts={'device': device}
)

In [ ]:
TEST_SIZE = 50 # for faster processing

with open('data/test_manifest.json', 'r') as f:
    test_manifest = json.load(f)
test_manifest = {
    k: v for k, v in list(test_manifest.items())[:TEST_SIZE]
}
# test_manifest

In [ ]:
true_dict = {key: test_manifest[key]['words'] for key in test_manifest}
# true_dict

In [ ]:
    ##############################
    #### YOUR CODE GOES BELOW #####
    ##############################

    ### Populate test_manifest with the first 50 test sentences and then call inference() below###

pred_dict = inference(our_model.to(device), test_manifest)

100%|██████████| 7/7 [23:46<00:00, 203.84s/it]


In [ ]:
details_by_utterance = sb.utils.edit_distance.wer_details_by_utterance(
    {k: v.split() for k, v in true_dict.items()},
    {k: v.split() for k, v in pred_dict.items()},
)

In [ ]:
sb.utils.edit_distance.wer_summary(details_by_utterance)

{'WER': 33.088235294117645,
 'SER': 52.0,
 'num_edits': 135,
 'num_scored_tokens': 408,
 'num_erraneous_sents': 26,
 'num_scored_sents': 50,
 'num_absent_sents': 0,
 'num_ref_sents': 50,
 'insertions': 94,
 'deletions': 9,
 'substitutions': 32}

In [ ]:
crdnn_pre_trained = EncoderDecoderASR.from_hparams(
    source='speechbrain/asr-crdnn-rnnlm-librispeech',
    savedir='asr-crdnn-rnnlm-librispeech',
    run_opts={'device': 'cuda'}
)

In [ ]:
pred_dict = inference(crdnn_pre_trained, test_manifest)

  0%|          | 0/7 [00:00<?, ?it/s]/usr/local/lib/python3.9/dist-packages/torch/functional.py:641: UserWarning: stft with return_complex=False is deprecated. In a future pytorch release, stft will return complex tensors for all inputs, and return_complex=False will raise an error.
Note: you can still call torch.view_as_real on the complex output to recover the old return format. (Triggered internally at ../aten/src/ATen/native/SpectralOps.cpp:862.)
  return _VF.stft(input, n_fft, hop_length, win_length, window,  # type: ignore[attr-defined]
100%|██████████| 7/7 [00:20<00:00,  2.96s/it]


In [ ]:
# this data structure stores WER information we use later. 
details_by_utterance = sb.utils.edit_distance.wer_details_by_utterance(
    {k: v.split() for k, v in true_dict.items()},
    {k: v.split() for k, v in pred_dict.items()},
)

In [ ]:
# word error rate (WER) summary using data structure we just created
sb.utils.edit_distance.wer_summary(details_by_utterance)

{'WER': 71.07843137254902,
 'SER': 92.0,
 'num_edits': 290,
 'num_scored_tokens': 408,
 'num_erraneous_sents': 46,
 'num_scored_sents': 50,
 'num_absent_sents': 0,
 'num_ref_sents': 50,
 'insertions': 108,
 'deletions': 28,
 'substitutions': 154}

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!gdown 1-s2e8dZYSjhVgfo_TL0V_89RZVhGnZ1Y #transcript.zip

!gdown 1ChdI1XyhmGq9z8Y8M38yXPMob6oPRqPO  #train.txt
!gdown 10w15DnUbJcQRBSZWP03qjM6Oq8l7WSVQ  #dev.txt


!gdown 1eimo-BFXZz6Z3FeZK8uC-wT84ji-JVos #hvb-audio.zip
!gdown https://cse.iitb.ac.in/~pjyothi/cs753/hvb-audio.zip
!unzip -q "/content/drive/MyDrive/hvb-audio.zip"

Downloading...
From: https://drive.google.com/uc?id=1-s2e8dZYSjhVgfo_TL0V_89RZVhGnZ1Y
To: /content/transcript.zip
100% 4.08M/4.08M [00:00<00:00, 173MB/s]
Access denied with the following error:

 	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses. 

You may still be able to access the file from the browser:

	 https://drive.google.com/uc?id=1eimo-BFXZz6Z3FeZK8uC-wT84ji-JVos 



In [ ]:
with open('transcript/370981f1f0254ebc.json', 'r') as f:
  print(f.read())

**1. Data Preprocessing**

In [ ]:
!pip install pydub
import os

def create_folder(folder_name):
  if not os.path.exists(folder_name):
      os.makedirs(folder_name)
      print(f"{folder_name} has been created.")
  else:
      print(f"{folder_name} already exists.")

create_folder("processed_data")
create_folder("sentiment_data")

# open the source file for reading
with open('train.yaml', 'r') as source_file:
    # read the contents of the source file
    file_contents = source_file.read()

# open the destination file for writing
with open('sentiment_train.yaml', 'w') as destination_file:
    # write the contents of the source file to the destination file
    destination_file.write(file_contents)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
processed_data already exists.
sentiment_data already exists.


In [ ]:
import os
import json
import pandas as pd
import soundfile as sf
from pydub import AudioSegment


def read_txt_file(file_path):
    with open(file_path, 'r') as f:
        lines = f.readlines()
    return lines


def extract_metadata_and_audio(lines, transcripts_path, audio_path,json_file):

    json_dict = {}

    for line in lines:
        conversation_id, utterance_indices = line.strip().split(':')
        utterance_indices = list(map(int, utterance_indices.split(',')))

        transcript_file = os.path.join(transcripts_path, f"{conversation_id}.json")
        agent_audio_file = os.path.join(audio_path, "agent", f"{conversation_id}.wav")
        caller_audio_file = os.path.join(audio_path, "caller", f"{conversation_id}.wav")

        with open(transcript_file, 'r') as f:
            transcript_data = json.load(f)

        agent_audio = AudioSegment.from_wav(agent_audio_file)
        caller_audio = AudioSegment.from_wav(caller_audio_file)

        for utterance in transcript_data:
            if utterance['index'] in utterance_indices:
                emotion = max(utterance['emotion'], key=utterance['emotion'].get)
                start_ms = utterance['start_ms']
                duration_ms = utterance['duration_ms']
                end_ms = start_ms + duration_ms
                relative_path=""
                if utterance['speaker_role'] == 'agent':
                    segment_audio = agent_audio[start_ms:end_ms]
                    relative_path = f"processed_data/{conversation_id}_{utterance['index']}_agent.wav"
                    segment_audio.export(f"processed_data/{conversation_id}_{utterance['index']}_agent.wav", format="wav")
                else:
                    segment_audio = caller_audio[start_ms:end_ms]
                    relative_path = f"processed_data/{conversation_id}_{utterance['index']}_caller.wav"
                    segment_audio.export(f"processed_data/{conversation_id}_{utterance['index']}_caller.wav", format="wav")



                json_dict[f"{conversation_id}_{utterance['index']}"] = {
                    "wav": relative_path,
                    "length": duration_ms/1000,
                    "words": utterance['transcript'],
                    "emotion" : emotion,
                    "speaker_role" : utterance['speaker_role']
                }

    # Writing the dictionary to the json file
    with open(json_file, mode="w") as json_f:
        json.dump(json_dict, json_f, indent=2)

    print(f"{json_file} successfully created!")

transcripts_path = '/content/transcript'
audio_path = '/content/audio'
train_txt = 'train.txt'
dev_txt = 'dev.txt'

train_lines = read_txt_file(train_txt)
dev_lines = read_txt_file(dev_txt)

extract_metadata_and_audio(train_lines, transcripts_path, audio_path,'/content/sentiment_data/train_manifest.json')
extract_metadata_and_audio(dev_lines, transcripts_path, audio_path,'/content/sentiment_data/dev_manifest.json')

/content/sentiment_data/train_manifest.json successfully created!
/content/sentiment_data/dev_manifest.json successfully created!


In [ ]:
from speechbrain.dataio.dataset import DynamicItemDataset
emotion_map = {"positive": 1, "neutral": 0, "negative": 2}

def loader(path,n=1e8):
    dataset = DynamicItemDataset.from_json(path)
    dataset = dataset.filtered_sorted(sort_key="length",select_n=n)
    dataset.add_dynamic_item(sb.dataio.dataio.read_audio, takes="wav", provides="signal")
    dataset.add_dynamic_item(lambda x: emotion_map[x], takes="emotion", provides="emotion_id")
    # dataset.set_output_keys(["id", "signal", "words", "tokens_list", "tokens_bos", "tokens_eos", "tokens"])
    dataset.set_output_keys(["id", "signal","emotion_id"])
    return dataset

dataset=loader(path="/content/sentiment_data/train_manifest.json")
dev_dataset = loader(path="/content/sentiment_data/dev_manifest.json")

In [ ]:
# print(len(dataset))
# print(len(dev_dataset))

In [ ]:
def count_labels(dataset):
    neg = 0
    pos = 0
    nu =  0
    err = 0
    for i in range(len(dataset)):
        if dataset[i]['emotion_id']==0: nu +=1
        elif dataset[i]['emotion_id']==1: pos+=1
        elif dataset[i]['emotion_id']==2: neg+=1
        else: err+=1

    print(f'positive = {pos}, negative = {neg}, neutral = {nu}, err = {err}')

In [ ]:
count_labels(dataset)
count_labels(dev_dataset)

positive = 2580, negative = 123, neutral = 5297, err = 0
positive = 1129, negative = 68, neutral = 2361, err = 0


In [ ]:
crdnn = EncoderDecoderASR.from_hparams(
    source='speechbrain/asr-crdnn-rnnlm-librispeech',
    savedir='asr-crdnn-rnnlm-librispeech',
    run_opts={'device': 'cpu'}
)

In [ ]:
modules = {
    "enc": crdnn.mods.encoder.model,                                   # extracting the encoder from the crdnn model 
    "compute_features": crdnn.mods.encoder.compute_features,           # extracting the comput_features from the crdnn model 
    "normalize": crdnn.mods.encoder.normalize,                         # extracting the normalize  from the crdnn model
    "classifier": torch.nn.Linear(encoder_output_size, num_emotions),  # initilizing a linear layer to add after the encoder
    "time_pool": torch.nn.AdaptiveAvgPool2d((1, encoder_output_size)), # adding a AdaptiveAvgPool2d() function to average pool to sequnce of outputs from the encoder blocks  
}

# Initialize weights using Xavier uniform initialization
torch.nn.init.xavier_uniform_(modules["classifier"].weight)
# Initialize biases to zero
torch.nn.init.zeros_(modules["classifier"].bias)

hparams = {
    "loss_fn": torch.nn.CrossEntropyLoss(),
    "log_softmax": sb.nnet.activations.Softmax(apply_log=True),
    "num_emotions": num_emotions,
    "valid_set" : dev_dataset,
    "batch_size" : 16
}


In [ ]:
import torch
import speechbrain as sb
from speechbrain.pretrained import EncoderDecoderASR
import torch.nn.functional as F


class EncEmotionClassifier(sb.Brain):
    def on_stage_start(self, stage, epoch):
        if stage == sb.Stage.TRAIN:
            for module in [self.modules.enc, self.modules.classifier]:
                for p in module.parameters():
                    p.requires_grad = True

    def compute_forward(self, batch, stage):
        batch = batch.to(self.device)
        wavs, wav_lens = batch.signal
        wavs, wav_lens = wavs.to(self.device), wav_lens.to(self.device)
        # Forward pass
        feats = self.modules.compute_features(wavs)
        feats = self.modules.normalize(feats, wav_lens)
        x = self.modules.enc(feats)
        # Pooling along the time dimension
        x = self.modules.time_pool(x)
        # Classifier
        logits = self.modules.classifier(x)
        probs = self.hparams.log_softmax(logits)

        return probs, logits, wav_lens

    def compute_objectives(self, predictions, batch, stage):
        probs, logits, _ = predictions
        logits = logits.squeeze(1)  # Remove the unnecessary dimension
        probs = probs.squeeze(1)    # Remove the unnecessary dimension
        labels = batch.emotion_id.to(self.device)
        print(f'labels = {labels}')
        loss = self.hparams.loss_fn(logits, labels)
        loss = torch.unsqueeze(loss, 0)
        print(f'loss = {type(loss)},{loss},{loss.shape}')
        preds = torch.argmax(probs, dim=-1)
        print(f'preds = {preds}')
        # Compute accuracy
        accuracy = (preds == labels).float().mean().item()
        print(f'batch accuracy = {accuracy}')
        return loss


    def fit_batch(self, batch):
        # print(batch)
        predictions = self.compute_forward(batch, sb.Stage.TRAIN)
        loss = self.compute_objectives(predictions, batch, sb.Stage.TRAIN)
        loss.backward()
        # Clip gradients
        torch.nn.utils.clip_grad_norm_(self.modules.parameters(), max_norm=1.0)
        if self.check_gradients(loss):
            self.optimizer.step()
        self.optimizer.zero_grad()
        return loss.detach()

    def on_stage_end(self, stage, stage_loss, epoch):
        super().on_stage_end(stage, epoch)
        if stage == sb.Stage.VALID:
            self.evaluate_emotion_classification()

    def evaluate_emotion_classification(self):
        # Set model to eval mode
        self.modules.eval()

        # Create a data loader for the validation set
        valid_set = self.hparams.valid_set
        valid_loader = sb.dataio.dataloader.make_dataloader(
            valid_set, batch_size=self.hparams.batch_size, num_workers=0)

        # Iterate over the validation set
        num_correct = 0
        num_total = 0
        for batch in valid_loader:
            with torch.no_grad():
                # Make predictions for the batch
                p_seq, _, _ = self.compute_forward(batch, sb.Stage.VALID)
                # Get the predicted emotion label for each example in the batch
                predicted_labels = torch.argmax(p_seq, dim=-1).squeeze()
                # Count the number of correct predictions
                num_correct += torch.sum(predicted_labels == batch.emotion_id).item()
                print(torch.sum(predicted_labels == batch.emotion_id).item())
                num_total += batch.emotion_id.size(0)
        # Compute the validation accuracy and print it
        valid_acc = num_correct / num_total
        print(f"Validation accuracy: {valid_acc:.2f}")

In [ ]:
# Create the emotion classifier model
num_emotions = 3  # Set the number of emotion classes you have
encoder_output_size = 512  # Set the output size of the encoder
brain = EncEmotionClassifier(modules, hparams=hparams, opt_class=lambda x: torch.optim.Adam(x, 1e-5))
brain.tokenizer = crdnn.tokenizer

In [ ]:
# Initialize an empty list to store accuracies
brain.epoch_valid_accuracies = []

#  Set up a training loop using the system object
brain.fit(
    epoch_counter=range(1),
    train_set= dataset,
    valid_set=dev_dataset,
    train_loader_kwargs={"shuffle": True, "batch_size": 16 },
    valid_loader_kwargs={"batch_size": 16},
    progressbar=True,
)

In [ ]:
# Initialize an empty list to store accuracies
brain.epoch_valid_accuracies = []

#  Set up a training loop using the system object
brain.fit(
    epoch_counter=range(1),
    train_set= dataset,
    valid_set=dev_dataset,
    train_loader_kwargs={"shuffle": True, "batch_size": 16 },
    valid_loader_kwargs={"batch_size": 16},
    progressbar=True,
)

  0%|          | 0/500 [00:00<?, ?it/s]

labels = tensor([0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0])
loss = <class 'torch.Tensor'>,tensor([1.2562], grad_fn=<UnsqueezeBackward0>),torch.Size([1])
preds = tensor([2, 2, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 2, 2, 2])
batch accuracy = 0.0


  0%|          | 1/500 [00:35<4:53:17, 35.27s/it, train_loss=1.26]

labels = tensor([0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0])
loss = <class 'torch.Tensor'>,tensor([1.2763], grad_fn=<UnsqueezeBackward0>),torch.Size([1])
preds = tensor([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 2, 2, 1, 2])
batch accuracy = 0.125


  0%|          | 2/500 [01:17<5:26:42, 39.36s/it, train_loss=1.27]

...

labels = tensor([0, 2, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0])
loss = <class 'torch.Tensor'>,tensor([0.5933], grad_fn=<UnsqueezeBackward0>),torch.Size([1])
preds = tensor([0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0])
batch accuracy = 0.875


 52%|█████▏    | 258/500 [2:36:52<1:53:50, 28.22s/it, train_loss=0.86]

labels = tensor([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1])
loss = <class 'torch.Tensor'>,tensor([0.6324], grad_fn=<UnsqueezeBackward0>),torch.Size([1])
preds = tensor([0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1])
batch accuracy = 0.8125


 52%|█████▏    | 259/500 [2:37:28<2:01:40, 30.29s/it, train_loss=0.859]

labels = tensor([0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0])
loss = <class 'torch.Tensor'>,tensor([0.6826], grad_fn=<UnsqueezeBackward0>),torch.Size([1])
preds = tensor([0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0])
batch accuracy = 0.75


**